# Coleta de dados

In [4]:
import pandas as pd
import os
import numpy as np
import yfinance as yf

from typing import Optional, List

import joblib

In [10]:
class AquisicaoDadosFundamentos:
    def __init__(self):
        self.balancos_dir = "dados/balancos/"
        self.balancos_joblib_file = "dados/fundamentos.joblib"
        self.fund_by_code = {}
        self.codes = self.get_code_list()

    def run(self):
        if os.path.isfile(self.balancos_joblib_file):
            print("carregando joblib fundamentos")
            return joblib.load(self.balancos_joblib_file)

        self.get_balancos_by_code()
        self.get_dre_by_code()

        self.salvar_joblib()
        
        return self.fund_by_code
    
    def get_code_list(self):
        return [file.replace("balanco_", "").replace(".xls", "") for file in os.listdir(self.balancos_dir)]

    def get_balancos_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)

        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            print(code)
            balanco = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=0)
            # colocar codigo na posicao 0, 0
            balanco.iloc[0, 0] = code
            # mudar coluna
            balanco.columns = balanco.iloc[0]
            balanco = balanco[1:]
            # tornar a 1ª coluna (que agora tem o nome da empresa)
            balanco = balanco.set_index(code)
            self.fund_by_code[code] = balanco

    def get_dre_by_code(self) -> None:

        files = os.listdir(self.balancos_dir)
        for file in files:
            code = file.replace("balanco_", "").replace(".xls", "")
            dre = pd.read_excel(f"{self.balancos_dir}{file}", sheet_name=1)
            # na primeira coluna colocar o título com o nome da empresa
            dre.iloc[0, 0] = code
            # pegar 1ª linha e tornar um cabeçalho
            dre.columns = dre.iloc[0]
            dre = dre[1:]
            # tornar a 1ª coluna (que agora tem o nome da empresa)
            dre = dre.set_index(code)
            self.fund_by_code[code] = self.fund_by_code[code].append(dre)
    
    def salvar_joblib(self):
        joblib.dump(self.fund_by_code, self.balancos_joblib_file)


In [12]:
fundamentos_by_code = AquisicaoDadosFundamentos().run()

carregando joblib fundamentos


In [16]:
fundamentos_by_code["ABEV3"].info()

<class 'pandas.core.frame.DataFrame'>
Index: 79 entries, Ativo Total to Lucro/Prejuízo do Período
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   31/12/2020  72 non-null     object
 1   30/09/2020  72 non-null     object
 2   30/06/2020  72 non-null     object
 3   31/03/2020  72 non-null     object
 4   31/12/2019  72 non-null     object
 5   30/09/2019  72 non-null     object
 6   30/06/2019  72 non-null     object
 7   31/03/2019  72 non-null     object
 8   31/12/2018  72 non-null     object
 9   30/09/2018  72 non-null     object
 10  30/06/2018  72 non-null     object
 11  31/03/2018  72 non-null     object
 12  31/12/2017  72 non-null     object
 13  30/09/2017  72 non-null     object
 14  30/06/2017  72 non-null     object
 15  31/03/2017  72 non-null     object
 16  31/12/2016  72 non-null     object
 17  30/09/2016  72 non-null     object
 18  30/06/2016  72 non-null     object
 19  31/03/2016  72 non-null 

In [17]:
def get_ibov_from_yahoo():
    
    ibov_file = "../dados/cotacoes_ibov.joblib"
    # Caso os dados ja tenham sido carregados anteriormente, retorne o joblib
    if os.path.isfile(ibov_file):
        print("carregando joblib ibov")
        return joblib.load(ibov_file)

    data_inicial = "2012-12-20"
    data_final = "2021-09-20"

    ibov = yf.download("^BVSP", start=data_inicial, end=data_final)

    # Salvar arquivo joblib para nao ter que rodar yfinance novamente     
    joblib.dump(ibov, ibov_file)

    return ibov

In [ ]:
def get_cotacoes_from_yahoo(codigos_acoes: List[str]):

    arquivo_cotacoes = "dados/Cotacoes.xlsx"
    # Caso os dados ja tenham sido carregados anteriormente, retorne o joblib
    if os.path.isfile(arquivo_cotacoes):
        cotacoes = pd.read_excel(arquivo_cotacoes, sheet_name=0):
        print("carregando cotacoes")
        return cotacoes

    data_inicial = "2012-12-20"
    data_final = "2021-09-20"

    cotacoes_yf = yf.download(codigos_acoes, start=data_inicial, end=data_final)
    
    cotacoes_yf.to_excel(cotacoes_yf, arquivo_cotacoes)
    
    return cotacoes_yf